# Demos: Lecture 16

## Demo 1: quiz 7 solution


<img src="fig/quiz_7_hamiltonian.png" width="450px"> 

In [1]:
import pennylane as qml
from pennylane import numpy as np

/opt/conda/envs/pennylane/lib/python3.8/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [2]:
coeffs = [1, 2, 3, -1, -2, -3]

obs = [
    qml.PauliX(0) @ qml.PauliX(1),
    qml.PauliX(1) @ qml.PauliX(2),
    qml.PauliX(0) @ qml.PauliX(2),
    qml.PauliZ(0),
    qml.PauliZ(1),
    qml.PauliZ(2)
]

H = qml.Hamiltonian(coeffs, obs)

In [3]:
print(H)

  (-3) [Z2]
+ (-2) [Z1]
+ (-1) [Z0]
+ (1) [X0 X1]
+ (2) [X1 X2]
+ (3) [X0 X2]


In [6]:
mat = np.zeros((8, 8))

for coeff, ob in zip(coeffs, obs):
    mat += coeff * qml.grouping.pauli_word_to_matrix(ob, wire_map={0:0, 1:1, 2:2})

In [7]:
mat

tensor([[-6.,  0.,  0.,  2.,  0.,  3.,  1.,  0.],
        [ 0.,  0.,  2.,  0.,  3.,  0.,  0.,  1.],
        [ 0.,  2., -2.,  0.,  1.,  0.,  0.,  3.],
        [ 2.,  0.,  0.,  4.,  0.,  1.,  3.,  0.],
        [ 0.,  3.,  1.,  0., -4.,  0.,  0.,  2.],
        [ 3.,  0.,  0.,  1.,  0.,  2.,  2.,  0.],
        [ 1.,  0.,  0.,  3.,  0.,  2.,  0.,  0.],
        [ 0.,  1.,  3.,  0.,  2.,  0.,  0.,  6.]], requires_grad=True)

In [8]:
eigvals, eigvecs = np.linalg.eigh(mat)

In [9]:
eigvals

array([-7.22832701, -5.79730808, -3.46850938, -1.93682438,  1.20997322,
        1.93682438,  7.22832701,  8.05584425])

In [10]:
eigvecs[:, 0]

tensor([-9.44761428e-01, -5.55111512e-17,  1.38777878e-17,
         1.44992997e-01,  5.06342000e-17,  2.93746984e-01,
        -1.07509707e-02, -5.66319226e-18], requires_grad=True)

<img src="fig/quiz_7_circuit.png" width="450px"> 

In [11]:
dev = qml.device('default.qubit', wires=3)

def ansatz(theta):
    qml.RY(theta[0], wires=0)
    qml.CNOT(wires=[0, 1])
    qml.CRY(theta[1], wires=[1, 2])
    qml.CNOT(wires=[2, 1])
    qml.CRY(theta[2], wires=[2, 1])
    qml.Toffoli(wires=[2, 1, 0])
    return qml.expval(H)

In [18]:
qnode = qml.QNode(ansatz, dev)

theta = np.array([0.0, 0.0, 0.0])

In [20]:
opt = qml.GradientDescentOptimizer(stepsize=0.05)

for _ in range(250):
    theta = opt.step(qnode, theta)

In [21]:
theta

tensor([-0.65149475,  3.01999151,  0.85866189], requires_grad=True)

In [22]:
qnode(theta)

tensor(-7.22248127, requires_grad=True)

## Demo 2: bit flip errors

In [ ]:
from lecture16_helpers import *
from scipy.stats import unitary_group

## Demo 3: depolarizing noise